In [32]:
Array= ['menus','menu/id','orders','orders/id','inventory','notifications','stores','loyality']

In [34]:
methods =[
    ['post','get'],
    ['get','put','delete'],
    ['post','get'],
    ['get','put','delete'],
    ['post','get'],
    ['post','get'],
    ['post','get'],
    ['post','get'],
          ]

In [35]:
tags =[
        'menu',
        'menu',
        'order',
        'order',
        'inventory',
        'notification',
        'store',
        'loyalty programs'

      ]

In [36]:
des = [
        ['Create a new menu','Finds all menus'],
        ['Returns a single menu','Update a menu in coffee shop','Deletes a menu'],
        ['Creates a new order','Finds all orders'],
        ['Returns a single order','Updates an order','Deletes an order'],
        ['Add a new Item to the inventory','Returns the inventory items'],
        ['Creates a new notification','Get all notifications'],
        ['Creates a new store','Finds all stores'],
        ['Creates a new loyalty program','Finds all loyalty programs'],
      ]

In [37]:
summ = [
        ['','Finds all menus'],
        ['Find menu by ID','Updates a menu','Deletes a menu'],
        ['Creates a new order','Finds all orders'],
        ['Find order by ID','Updates an order','Deletes an order'],
        ['Add a new Item to the inventory','Updates an order','Returns the inventory items'],
        ['Creates a new notification',' Get all notifications'],
        ['Creates a new store',' Finds all stores'],
        ['Creates a new loyalty program','Finds all loyalty programs'],
      
         
      ]

In [38]:
empty_indices = []  # To store the indices of empty lists

In [39]:
for i in range(len(Array)):
    if (des[i][0]) == '' and (summ[i][0]) == '':
        empty_indices.append(i)

In [40]:
for i in range(len(empty_indices)):
    for x in range(len(methods[i])):
        
        if methods[empty_indices[i]][x] == 'get':
               
               des[empty_indices[i]][x] = 'get'+ ' ' + Array[empty_indices[i]]+ ' ' + 'details'

        elif methods[empty_indices[i]][x] == 'post':
               
               des[empty_indices[i]][x] = 'add'+ ' '  + Array[i] + ' ' + 'details'
               
        elif methods[empty_indices[i]][x] == 'put':
                            
                des[empty_indices[i]][x] = 'update'+ ' '  + Array[i] + ' ' + 'details'
                 
        elif methods[empty_indices[i]][x] == 'delete':
               
                des[empty_indices[i]][x] = 'delete'+ ' '  + Array[i] +  ' ' + 'details'
             
        else: 
             
                des[i][x] =Array[i] 
              

In [41]:
appenddocs = [' , '.join(sublist) for sublist in des]

In [42]:
appenddocs_summery = [' , '.join(sublist) for sublist in summ]

In [43]:
full_doc=[]

In [44]:
for i in range(len(Array)):
    # Concatenate corresponding elements from first and second lists
    full_doc.append({"path_name": Array[i] ,"description": appenddocs[i], "summery": appenddocs_summery[i] })


In [48]:
full_doc

[{'path_name': 'menus',
  'description': 'Create a new menu , Finds all menus',
  'summery': ' , Finds all menus'},
 {'path_name': 'menu/id',
  'description': 'Returns a single menu , Update a menu in coffee shop , Deletes a menu',
  'summery': 'Find menu by ID , Updates a menu , Deletes a menu'},
 {'path_name': 'orders',
  'description': 'Creates a new order , Finds all orders',
  'summery': 'Creates a new order , Finds all orders'},
 {'path_name': 'orders/id',
  'description': 'Returns a single order , Updates an order , Deletes an order',
  'summery': 'Find order by ID , Updates an order , Deletes an order'},
 {'path_name': 'inventory',
  'description': 'Add a new Item to the inventory , Returns the inventory items',
  'summery': 'Add a new Item to the inventory , Updates an order , Returns the inventory items'},
 {'path_name': 'notifications',
  'description': 'Creates a new notification , Get all notifications',
  'summery': 'Creates a new notification ,  Get all notifications'},


In [49]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings
sentence_embeddings = model.encode(full_doc)
sentence_embeddings.shape

(8, 768)

In [50]:
import faiss

In [51]:
d = sentence_embeddings.shape[1]
index = faiss.IndexFlatIP(d)

In [52]:
index.is_trained

True

In [53]:
index.add(sentence_embeddings)

In [54]:
import numpy as np
from numpy.linalg import norm

In [55]:
def Getting_relevent_path(sentence):
    k = 1
    xq = model.encode([sentence])
    D, I = index.search(xq, k)

    get_cosine_value =  Get_cosine(xq,I)
    Relevent_path = Array[I[0][0]]
    Tag = tags[I[0][0]]

    return get_cosine_value,Relevent_path,Tag

In [56]:
def Get_cosine(xq,I):
    cosine = np.dot(sentence_embeddings[I[0]], xq[0]) / (np.linalg.norm(sentence_embeddings[I[0]]) * np.linalg.norm(xq[0]))
    cosine_value = cosine[0]  # Extracting the value from the array

    return cosine_value

In [57]:
def New_user_Method(user_path):
     if user_path in Array:
         found = False
         for index, path in enumerate(Array):
               if user_path == path:
                    found = True
                    path_index = index
                    tag = tags[path_index] 
                    break
         

     else:
          Array.append(user_path)
          tags.append('Default')
          path_index= int(len(Array))-1
          tag = tags[path_index] 

     return user_path,tag




c:\Users\Admin\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--sentence-transformers--bert-base-nli-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [74]:
#import spacy
#import pytextrank

#nlp= spacy.load("en_core_web_lg")

#nlp.add_pipe("textrank")

In [75]:
# def suggesting_name(sentence):
#         doc=nlp(sentence)
#         phrases_and_ranks = [ 
#                     (phrase.chunks[0]) for phrase in doc._.phrases
#         ]

#         if(len(phrases_and_ranks[0])<=3):
#                             path = phrases_and_ranks[0]
#         else:
#                             path = phrases_and_ranks[0][2:]
#         path= path.text.replace(" ", "_")
#         return path

In [76]:
# name = suggesting_name('Give me a code to update food details')
# name

Error during conversion: ValueError('Queue is full! Please try again.')
Exception in thread Thread-autoconversion:
Traceback (most recent call last):
  File "c:\Users\Admin\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\Admin\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Admin\anaconda3\Lib\site-packages\transformers\safetensors_conversion.py", line 89, in auto_conversion
    sha = get_conversion_pr_reference(api, pretrained_model_name_or_path, **cached_file_kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\anaconda3\Lib\site-packages\transformers\safetensors_conversion.py", line 82, in get_conversion_pr_reference
    sha = f"refs/pr/{pr.num}"
                     ^^^^^^
AttributeError: 'NoneType' object has no attribute 'num'
